In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import log_loss
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn import ensemble, metrics, model_selection as ms, preprocessing, tree
from sklearn import linear_model as lm
import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.set_option('display.max_columns', 50)
from sklearn.pipeline import Pipeline
from os import listdir
from os.path import isfile, join

In [49]:
data_folder = "C:/Users/Byron/Documents/Football Predictions/UnderstatPredictions/Datasets"

In [61]:
pred_df = pd.read_csv(data_folder+"/xg_predictions.csv")
pred_df.head()

,league,season,fixture,season_gw,Team_Home,Team_Away,homegoals,awaygoals,Result,AverageMethod,xG_av_Home,xGA_av_Home,AttStr_Home,DefStr_Home,xG_av_Away,xGA_av_Away,AttStr_Away,DefStr_Away,SeasonHG,SeasonAG,Pred_HG,Pred_AG,Prob_HG,Prob_AG,score,HomeProb,DrawProb,AwayProb,PredScore,PredScoreProb,PredResult,PredResultProb,PredResultScore,PredResultScoreProb,FinalResultProb,FinalScore,PredCorrect,PredResultScoreCorrect,PredScoreCorrect
0,Bundesliga,2014/15,Bayern Munich vs Wolfsburg,1,Bayern Munich,Wolfsburg,2.0,1.0,H,EMA No reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan nan nan nan nan nan nan nan nan nan],[nan nan nan nan nan nan nan nan nan nan],0 1 2 3 4 5 6 7 8 9\n0 N...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 . 1,0,0,0
1,Bundesliga,2014/15,Bayern Munich vs Wolfsburg,1,Bayern Munich,Wolfsburg,2.0,1.0,H,Moving Average No reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan nan nan nan nan nan nan nan nan nan],[nan nan nan nan nan nan nan nan nan nan],0 1 2 3 4 5 6 7 8 9\n0 N...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 . 1,0,0,0
2,Bundesliga,2014/15,Bayern Munich vs Wolfsburg,1,Bayern Munich,Wolfsburg,2.0,1.0,H,EMA Season reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan nan nan nan nan nan nan nan nan nan],[nan nan nan nan nan nan nan nan nan nan],0 1 2 3 4 5 6 7 8 9\n0 N...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 . 1,0,0,0
3,Bundesliga,2014/15,Bayern Munich vs Wolfsburg,1,Bayern Munich,Wolfsburg,2.0,1.0,H,Moving Average Season reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan nan nan nan nan nan nan nan nan nan],[nan nan nan nan nan nan nan nan nan nan],0 1 2 3 4 5 6 7 8 9\n0 N...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 . 1,0,0,0
4,Bundesliga,2014/15,Hoffenheim vs Augsburg,1,Hoffenheim,Augsburg,2.0,0.0,H,EMA No reset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[nan nan nan nan nan nan nan nan nan nan],[nan nan nan nan nan nan nan nan nan nan],0 1 2 3 4 5 6 7 8 9\n0 N...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 . 0,0,0,0


In [51]:
bookie_df =pd.read_csv(data_folder+"/bookie_odds.csv").drop(columns="Unnamed: 0")
bookie_df.head()

,league,season,Date,fixture,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,BookieHomeOdds,BookieDrawOdds,BookieAwayOdds,bookie,BookieHomeProb,BookieDrawProb,BookieAwayProb,BookieMargin,BookiePredResult,BookiePredResultProb,BookieFinalResultProb,BookiePredCorrect
0,EPL,2014/15,16/08/14,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,1.0,H,1.0,1.0,D,1.25,6.50,15.0,Bet365,0.800000,0.153846,0.066667,2.051282,H,0.800000,0.800000,1
1,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.5,Bet365,0.751880,0.173913,0.105263,3.105590,H,0.751880,0.173913,0
2,EPL,2016/17,01/01/17,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,0.0,H,1.0,0.0,H,1.29,5.50,9.5,Bet365,0.775194,0.181818,0.105263,6.227514,H,0.775194,0.775194,1
3,EPL,2017/18,20/01/2018,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,4.0,1.0,H,4.0,0.0,H,1.50,4.75,7.0,Bet365,0.666667,0.210526,0.142857,2.005013,H,0.666667,0.666667,1
4,EPL,2018/19,21/04/2019,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,3.0,A,0.0,1.0,A,1.53,4.60,6.5,Bet365,0.653595,0.217391,0.153846,2.483223,H,0.653595,0.153846,0


In [62]:
pred_df = pred_df[["league","season","fixture","season_gw","Team_Home","Team_Away","Result","AverageMethod","HomeProb","DrawProb","AwayProb"
                 ,"PredResult","PredScore","PredScoreProb","PredResultScore","PredResultScoreProb","FinalScore"
                 ,"PredCorrect","PredResultScoreCorrect","PredScoreCorrect"]]
gambling_df = pd.merge(bookie_df,pred_df,on=["league","season","fixture"])
print(gambling_df.head(20))

   league   season      Date                    fixture HomeTeam  \
0     EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
1     EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
2     EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
3     EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
4     EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
5     EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
6     EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
7     EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
8     EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
9     EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
10    EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
11    EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
12    EPL  2014/15  16/08/14  Arsenal vs Crystal Palace  Arsenal   
13    EPL  2014/15  16/08/14  Arsenal vs Crystal

In [63]:
#define stake
stake = 5

def returns(stake,odds):
    returns = stake*odds-stake
    return returns

def expected_returns(stake,returns,prob):
    p = prob/100
    exp_returns = (returns*p)-((1-p)*stake)
    return exp_returns

def result_returns(row):
    #if np.isnan(row["PredResult"])==False:
    if row["Result"] == "H":
        return row["HomeReturns"]
    if row["Result"] == "D":
        return row["DrawReturns"]
    if row["Result"] == "A":
        return row["AwayReturns"]

gambling_df["Stake"] = stake
gambling_df["HomeReturns"]=gambling_df["BookieHomeOdds"].apply(lambda x: returns(stake,x))
gambling_df["DrawReturns"]=gambling_df["BookieDrawOdds"].apply(lambda x: returns(stake,x))
gambling_df["AwayReturns"]=gambling_df["BookieAwayOdds"].apply(lambda x: returns(stake,x))

gambling_df["ResultReturns"]= gambling_df.apply(lambda row: result_returns(row), axis =1)

gambling_df["ExpHomeReturns"] = gambling_df.apply(lambda row: expected_returns(row.Stake,row.HomeReturns,row.HomeProb)
                                                , axis = 1)
gambling_df["ExpDrawReturns"] = gambling_df.apply(lambda row: expected_returns(row.Stake,row.DrawReturns,row.DrawProb)
                                                , axis = 1)
gambling_df["ExpAwayReturns"] = gambling_df.apply(lambda row: expected_returns(row.Stake,row.AwayReturns,row.AwayProb)
                                                , axis = 1)
gambling_df[gambling_df["season_gw"]>2].head()

    
    

,league,season,Date,fixture,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,BookieHomeOdds,BookieDrawOdds,BookieAwayOdds,bookie,BookieHomeProb,BookieDrawProb,BookieAwayProb,BookieMargin,BookiePredResult,BookiePredResultProb,BookieFinalResultProb,BookiePredCorrect,season_gw,Team_Home,Team_Away,Result,AverageMethod,HomeProb,DrawProb,AwayProb,PredResult,PredScore,PredScoreProb,PredResultScore,PredResultScoreProb,FinalScore,PredCorrect,PredResultScoreCorrect,PredScoreCorrect,Stake,HomeReturns,DrawReturns,AwayReturns,ResultReturns,ExpHomeReturns,ExpDrawReturns,ExpAwayReturns
20,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,Arsenal,Crystal Palace,D,EMA No reset,67.730937,19.076814,13.183893,H,2 . 0,11.780507,2 . 0,11.780507,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.495893,0.484584,1.262349
21,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,Arsenal,Crystal Palace,D,Moving Average No reset,71.265125,17.469369,11.251352,H,2 . 0,12.045066,2 . 0,12.045066,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.260869,0.022444,0.344392
22,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,Arsenal,Crystal Palace,D,EMA Season reset,68.519684,18.641712,12.828669,H,2 . 0,11.693262,2 . 0,11.693262,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.443441,0.359492,1.093618
23,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,Arsenal,Crystal Palace,D,Moving Average Season reset,71.800405,17.143965,11.039495,H,2 . 0,11.919160,2 . 0,11.919160,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.225273,-0.071110,0.243760
24,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.00,8.75,BetWin,0.75188,0.200000,0.114286,6.616541,H,0.75188,0.200000,0,34,Arsenal,Crystal Palace,D,EMA No reset,67.730937,19.076814,13.183893,H,2 . 0,11.780507,2 . 0,11.780507,1 . 1,0,0,0,5,1.65,20.00,38.75,20.00,-0.495893,-0.230796,0.767953


In [64]:
#def determine_bet()

gambling_df["BestValueExpReturns"]= gambling_df[["ExpHomeReturns","ExpDrawReturns","ExpAwayReturns"]].max(axis=1)


def determine_bet(row):
    #if np.isnan(row["PredResult"])==False:
    if row["BestValueExpReturns"] == row["ExpHomeReturns"]:
        return "H"
    if row["BestValueExpReturns"] == row["ExpDrawReturns"]:
        return "D"
    if row["BestValueExpReturns"] == row["ExpAwayReturns"]:
        return "A"

gambling_df["ValuePredResult"] = gambling_df.apply(lambda row: determine_bet(row),axis = 1)
gambling_df[gambling_df["season_gw"]>2].head()

,league,season,Date,fixture,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,BookieHomeOdds,BookieDrawOdds,BookieAwayOdds,bookie,BookieHomeProb,BookieDrawProb,BookieAwayProb,BookieMargin,BookiePredResult,BookiePredResultProb,BookieFinalResultProb,BookiePredCorrect,season_gw,...,Team_Away,Result,AverageMethod,HomeProb,DrawProb,AwayProb,PredResult,PredScore,PredScoreProb,PredResultScore,PredResultScoreProb,FinalScore,PredCorrect,PredResultScoreCorrect,PredScoreCorrect,Stake,HomeReturns,DrawReturns,AwayReturns,ResultReturns,ExpHomeReturns,ExpDrawReturns,ExpAwayReturns,BestValueExpReturns,ValuePredResult
20,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,...,Crystal Palace,D,EMA No reset,67.730937,19.076814,13.183893,H,2 . 0,11.780507,2 . 0,11.780507,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.495893,0.484584,1.262349,1.262349,A
21,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,...,Crystal Palace,D,Moving Average No reset,71.265125,17.469369,11.251352,H,2 . 0,12.045066,2 . 0,12.045066,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.260869,0.022444,0.344392,0.344392,A
22,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,...,Crystal Palace,D,EMA Season reset,68.519684,18.641712,12.828669,H,2 . 0,11.693262,2 . 0,11.693262,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.443441,0.359492,1.093618,1.093618,A
23,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,...,Crystal Palace,D,Moving Average Season reset,71.800405,17.143965,11.039495,H,2 . 0,11.919160,2 . 0,11.919160,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.225273,-0.071110,0.243760,0.243760,A
24,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.00,8.75,BetWin,0.75188,0.200000,0.114286,6.616541,H,0.75188,0.200000,0,34,...,Crystal Palace,D,EMA No reset,67.730937,19.076814,13.183893,H,2 . 0,11.780507,2 . 0,11.780507,1 . 1,0,0,0,5,1.65,20.00,38.75,20.00,-0.495893,-0.230796,0.767953,0.767953,A


In [68]:
gambling_df["ValuePredCorrect"] = np.where(gambling_df["Result"]==gambling_df["ValuePredResult"],1,0)
gambling_df["ValuePredReturns"] = np.where(gambling_df["ValuePredCorrect"]==1,
                                           gambling_df["ResultReturns"],gambling_df["Stake"]*-1)
gambling_df[gambling_df["season_gw"]>2].head()

,league,season,Date,fixture,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,BookieHomeOdds,BookieDrawOdds,BookieAwayOdds,bookie,BookieHomeProb,BookieDrawProb,BookieAwayProb,BookieMargin,BookiePredResult,BookiePredResultProb,BookieFinalResultProb,BookiePredCorrect,season_gw,...,AverageMethod,HomeProb,DrawProb,AwayProb,PredResult,PredScore,PredScoreProb,PredResultScore,PredResultScoreProb,FinalScore,PredCorrect,PredResultScoreCorrect,PredScoreCorrect,Stake,HomeReturns,DrawReturns,AwayReturns,ResultReturns,ExpHomeReturns,ExpDrawReturns,ExpAwayReturns,BestValueExpReturns,ValuePredResult,ValuePredCorrect,ValuePredReturns
20,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,...,EMA No reset,67.730937,19.076814,13.183893,H,2 . 0,11.780507,2 . 0,11.780507,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.495893,0.484584,1.262349,1.262349,A,0,-5.0
21,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,...,Moving Average No reset,71.265125,17.469369,11.251352,H,2 . 0,12.045066,2 . 0,12.045066,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.260869,0.022444,0.344392,0.344392,A,0,-5.0
22,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,...,EMA Season reset,68.519684,18.641712,12.828669,H,2 . 0,11.693262,2 . 0,11.693262,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.443441,0.359492,1.093618,1.093618,A,0,-5.0
23,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.50,Bet365,0.75188,0.173913,0.105263,3.105590,H,0.75188,0.173913,0,34,...,Moving Average Season reset,71.800405,17.143965,11.039495,H,2 . 0,11.919160,2 . 0,11.919160,1 . 1,0,0,0,5,1.65,23.75,42.50,23.75,-0.225273,-0.071110,0.243760,0.243760,A,0,-5.0
24,EPL,2015/16,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.00,8.75,BetWin,0.75188,0.200000,0.114286,6.616541,H,0.75188,0.200000,0,34,...,EMA No reset,67.730937,19.076814,13.183893,H,2 . 0,11.780507,2 . 0,11.780507,1 . 1,0,0,0,5,1.65,20.00,38.75,20.00,-0.495893,-0.230796,0.767953,0.767953,A,0,-5.0


In [76]:
qa_df = gambling_df[(gambling_df["bookie"]=="Bet365") &
                   (gambling_df["BestValueExpReturns"]>0)].copy()
bins = pd.IntervalIndex.from_tuples([(-1, 0), (0.01, 0.5), (0.501, 2),(2.001,5),(5.001,10),(10.001,100)])
qa_df['bet_bin'] = pd.cut(qa_df['BestValueExpReturns'],bins)

qa_df.dropna().groupby(["AverageMethod","league","season","bet_bin"]).agg({"ValuePredCorrect":"mean"
                                                                         ,"ValuePredReturns":"sum"
                                                                          ,"fixture":"nunique"}).reset_index().head(60)



,AverageMethod,league,season,bet_bin,ValuePredCorrect,ValuePredReturns,fixture
0,EMA No reset,Bundesliga,2014/15,"(-1.0, 0.0]",NaN,NaN,NaN
1,EMA No reset,Bundesliga,2014/15,"(0.01, 0.5]",0.351648,-68.10,91.0
2,EMA No reset,Bundesliga,2014/15,"(0.501, 2.0]",0.300813,-61.95,123.0
3,EMA No reset,Bundesliga,2014/15,"(2.001, 5.0]",0.200000,18.05,35.0
4,EMA No reset,Bundesliga,2014/15,"(5.001, 10.0]",0.375000,46.00,8.0
5,EMA No reset,Bundesliga,2014/15,"(10.001, 100.0]",0.000000,-10.00,2.0
6,EMA No reset,Bundesliga,2015/16,"(-1.0, 0.0]",NaN,NaN,NaN
7,EMA No reset,Bundesliga,2015/16,"(0.01, 0.5]",0.426966,46.55,89.0
8,EMA No reset,Bundesliga,2015/16,"(0.501, 2.0]",0.258993,-137.50,139.0
9,EMA No reset,Bundesliga,2015/16,"(2.001, 5.0]",0.172414,12.00,29.0


In [77]:
qa_df.dropna().groupby(["league","AverageMethod","bet_bin"]).agg({"ValuePredCorrect":"mean"
                                                                         ,"ValuePredReturns":["sum","mean"]
                                                                 ,"fixture":"nunique"}).reset_index().head(60)

league                AverageMethod          bet_bin ValuePredCorrect  \
                                                                         mean   
0   Bundesliga                 EMA No reset      (-1.0, 0.0]              NaN   
1   Bundesliga                 EMA No reset      (0.01, 0.5]         0.398524   
2   Bundesliga                 EMA No reset     (0.501, 2.0]         0.260989   
3   Bundesliga                 EMA No reset     (2.001, 5.0]         0.197861   
4   Bundesliga                 EMA No reset    (5.001, 10.0]         0.120000   
5   Bundesliga                 EMA No reset  (10.001, 100.0]         0.250000   
6   Bundesliga             EMA Season reset      (-1.0, 0.0]              NaN   
7   Bundesliga             EMA Season reset      (0.01, 0.5]         0.406852   
8   Bundesliga             EMA Season reset     (0.501, 2.0]         0.303519   
9   Bundesliga             EMA Season reset     (2.001, 5.0]         0.273764   
10  Bundesliga             EMA Season reset    (5.001, 10.0]         0.304348   
11  Bundesliga             EMA Season reset  (10.001, 100.0]         0.000000   
12  Bundesliga      Moving Average No reset      (-1.0, 0.0]              NaN   
13  Bundesliga      Moving Average No reset      (0.01, 0.5]         0.337143   
14  Bundesliga      Moving Average No reset     (0.501, 2.0]         0.271370   
15  Bundesliga      Moving Average No reset     (2.001, 5.0]         0.219048   
16  Bundesliga      Moving Average No reset    (5.001, 10.0]         0.076923   
17  Bundesliga      Moving Average No reset  (10.001, 100.0]         0.250000   
18  Bundesliga  Moving Average Season reset      (-1.0, 0.0]              NaN   
19  Bundesliga  Moving Average Season reset      (0.01, 0.5]         0.394000   
20  Bundesliga  Moving Average Season reset     (0.501, 2.0]         0.328221   
21  Bundesliga  Moving Average Season reset     (2.001, 5.0]         0.255725   
22  Bundesliga  Moving Average Season reset    (5.001, 10.0]         0.279070   
23  Bundesliga  Moving Average Season reset  (10.001, 100.0]         0.000000   
24         EPL                 EMA No reset      (-1.0, 0.0]              NaN   
25         EPL                 EMA No reset      (0.01, 0.5]         0.354344   
26         EPL                 EMA No reset     (0.501, 2.0]         0.289221   
27         EPL                 EMA No reset     (2.001, 5.0]         0.193989   
28         EPL                 EMA No reset    (5.001, 10.0]         0.106667   
29         EPL                 EMA No reset  (10.001, 100.0]         0.416667   
30         EPL             EMA Season reset      (-1.0, 0.0]              NaN   
31         EPL             EMA Season reset      (0.01, 0.5]         0.397459   
32         EPL             EMA Season reset     (0.501, 2.0]         0.312918   
33         EPL             EMA Season reset     (2.001, 5.0]         0.227749   
34         EPL             EMA Season reset    (5.001, 10.0]         0.137255   
35         EPL             EMA Season reset  (10.001, 100.0]         0.312500   
36         EPL      Moving Average No reset      (-1.0, 0.0]              NaN   
37         EPL      Moving Average No reset      (0.01, 0.5]         0.406872   
38         EPL      Moving Average No reset     (0.501, 2.0]         0.304942   
39         EPL      Moving Average No reset     (2.001, 5.0]         0.168798   
40         EPL      Moving Average No reset    (5.001, 10.0]         0.141026   
41         EPL      Moving Average No reset  (10.001, 100.0]         0.444444   
42         EPL  Moving Average Season reset      (-1.0, 0.0]              NaN   
43         EPL  Moving Average Season reset      (0.01, 0.5]         0.402460   
44         EPL  Moving Average Season reset     (0.501, 2.0]         0.313836   
45         EPL  Moving Average Season reset     (2.001, 5.0]         0.226776   
46         EPL  Moving Average Season reset    (5.001, 10.0]         0.170213   
47         EPL  Moving Average Season r

In [80]:
qa_df.dropna().groupby(["bet_bin","season"]).agg({"ValuePredCorrect":"mean"
                                                                         ,"ValuePredReturns":["sum","mean"]
                                                                 ,"fixture":"nunique"}).reset_index().head(60)

bet_bin   season ValuePredCorrect ValuePredReturns             \
                                         mean              sum       mean   
0       (-1.0, 0.0]  2014/15              NaN              NaN        NaN   
1       (-1.0, 0.0]  2015/16              NaN              NaN        NaN   
2       (-1.0, 0.0]  2016/17              NaN              NaN        NaN   
3       (-1.0, 0.0]  2017/18              NaN              NaN        NaN   
4       (-1.0, 0.0]  2018/19              NaN              NaN        NaN   
5       (-1.0, 0.0]  2019/20              NaN              NaN        NaN   
6       (0.01, 0.5]  2014/15         0.351406         -1614.50  -0.810492   
7       (0.01, 0.5]  2015/16         0.390336          -686.35  -0.325130   
8       (0.01, 0.5]  2016/17         0.412823          -932.45  -0.430097   
9       (0.01, 0.5]  2017/18         0.359525         -1143.80  -0.522522   
10      (0.01, 0.5]  2018/19         0.378352          -529.80  -0.258313   
11      (0.01, 0.5]  2019/20         0.392453           -80.05  -0.060415   
12     (0.501, 2.0]  2014/15         0.311945          -629.50  -0.214846   
13     (0.501, 2.0]  2015/16         0.281072         -2003.00  -0.671022   
14     (0.501, 2.0]  2016/17         0.313253         -1853.85  -0.620432   
15     (0.501, 2.0]  2017/18         0.277010          -986.65  -0.349504   
16     (0.501, 2.0]  2018/19         0.265679         -1589.25  -0.547640   
17     (0.501, 2.0]  2019/20         0.319869          -245.55  -0.134034   
18     (2.001, 5.0]  2014/15         0.218692          -305.20  -0.285234   
19     (2.001, 5.0]  2015/16         0.216463          -336.90  -0.342378   
20     (2.001, 5.0]  2016/17         0.202756          -334.30  -0.329035   
21     (2.001, 5.0]  2017/18         0.161725         -1036.80  -0.931536   
22     (2.001, 5.0]  2018/19         0.186047           346.20   0.298191   
23     (2.001, 5.0]  2019/20         0.192357           602.15   0.767070   
24    (5.001, 10.0]  2014/15         0.181818           -29.00  -0.119835   
25    (5.001, 10.0]  2015/16         0.189655           327.00   1.879310   
26    (5.001, 10.0]  2016/17         0.116402          -380.50  -2.013228   
27    (5.001, 10.0]  2017/18         0.125628           141.50   0.711055   
28    (5.001, 10.0]  2018/19         0.119691           -57.70  -0.222780   
29    (5.001, 10.0]  2019/20         0.179310           297.70   2.053103   
30  (10.001, 100.0]  2014/15         0.333333           530.00  14.722222   
31  (10.001, 100.0]  2015/16         0.133333            38.30   0.851111   
32  (10.001, 100.0]  2016/17         0.000000          -170.00  -5.000000   
33  (10.001, 100.0]  2017/18         0.074074            55.00   2.037037   
34  (10.001, 100.0]  2018/19         0.025641          -100.00  -2.564103   
35  (10.001, 100.0]  2019/20         0.120000           162.50   6.500000   

   fixture  
   nunique  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN  
5      NaN  
6    693.0  
7   1021.0  
8   1052.0  
9   1040.0  
10   976.0  
11   690.0  
12   920.0  
13  1241.0  
14  1292.0  
15  1221.0  
16  1228.0  
17   837.0  
18   326.0  
19   453.0  
20   467.0  
21   511.0  
22   509.0  
23   377.0  
24    69.0  
25    88.0  
26    87.0  
27    99.0  
28   110.0  
29    77.0  
30    10.0  
31    20.0  
32    15.0  
33    13.0  
34    18.0  
35    13.0